In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Epoch 공모전/2020_2023_최종데이터.csv')
df.drop(['tm_dt'], axis=1, inplace=True)

In [ ]:
# 범주형 변수 인코딩
df = pd.get_dummies(df, columns=["address_city", "address_gu", "sub_address"])

In [ ]:
# X, y 분리
X = df.drop("call_count", axis=1)
y = df["call_count"]

# 학습/테스트 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 정규화 (StandardScaler)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 모델 구성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import LeakyReLU

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# 컴파일
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# 콜백 설정
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 학습
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 3.6997 - mae: 1.0644 - val_loss: 2.3059 - val_mae: 0.8716
Epoch 2/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.8347 - mae: 0.9157 - val_loss: 2.0851 - val_mae: 0.8478
Epoch 3/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 4.2997 - mae: 0.9110 - val_loss: 1.9414 - val_mae: 0.8262
Epoch 4/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.5329 - mae: 0.8710 - val_loss: 1.9694 - val_mae: 0.8051
Epoch 5/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3.0829 - mae: 0.8635 - val_loss: 1.9547 - val_mae: 0.8256
Epoch 6/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.8149 - mae: 0.8267 - val_loss: 2.5288 - val_mae: 0.8347
Epoch 7/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.3167 - mae: 0.8440 - val_loss: 1.9660 - val_mae: 0.8135
Epoch 8/50
859/859 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.2390 - mae: 0.8313 - val_loss: 2.0232 - val_mae: 0.8185


In [ ]:
# 검증 데이터 로드
valid_df = pd.read_csv("/content/drive/MyDrive/Epoch 공모전/2024_최종데이터.csv")

# 훈련 데이터에는 없던 열 제거
valid_df.drop(['tm_dt'], axis=1, inplace=True)

# X 분리
X_valid = valid_df.drop(columns=['call_count'])  # 예측 대상 제외

# 인코딩: 훈련과 동일하게 One-Hot
X_valid = pd.get_dummies(X_valid)

# 누락된 열 보완 및 순서 맞추기
X_valid = X_valid.reindex(columns=X_train.columns, fill_value=0)

# 정규화 (훈련 스케일러 재사용)
X_valid_scaled = scaler.transform(X_valid)

# 예측
y_valid_pred = model.predict(X_valid_scaled)
y_valid_pred = np.round(y_valid_pred).astype(int).flatten()

# 예측값 덮어쓰기
valid_df['call_count'] = y_valid_pred

# 상위 결과 확인
from IPython.display import display
display(valid_df[['call_count']].head(10))

301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


,call_count
0,1
1,1
2,2
3,1
4,1
5,1
6,1
7,1
8,2
9,2


In [ ]:
result_df = pd.read_csv('/content/drive/MyDrive/Epoch 공모전/test_call119.csv', encoding='euc-kr')

In [ ]:
result_df['call_count'] = valid_df['call_count']

In [ ]:
result_df.head()

,TM,address_city,address_gu,sub_address,STN,ta_max,ta_min,ta_max_min,hm_min,hm_max,ws_max,ws_ins_max,rn_day,call_count
0,20240501,부산광역시,강서구,대저2동,904,18.5,11.1,7.4,42.5,82.5,6.5,11.6,0.0,1
1,20240501,부산광역시,강서구,생곡동,904,18.5,11.1,7.4,42.5,82.5,6.5,11.6,0.0,1
2,20240501,부산광역시,강서구,송정동,937,16.9,9.9,7.0,55.3,93.9,4.5,9.7,0.0,2
3,20240501,부산광역시,강서구,신호동,950,16.6,11.4,5.2,48.1,84.6,6.4,13.5,0.0,1
4,20240501,부산광역시,금정구,구서동,940,16.9,10.2,6.7,46.8,91.3,3.3,8.7,0.0,1


In [ ]:
result_df.to_csv('/content/drive/MyDrive/Epoch 공모전/250028.csv', index=False, encoding='euc-kr')